In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2021/train.csv")
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
print('# of features = ', len(train_df.columns), '\n')
train_df.columns

# 100 features, drop 'id', TV is 'loss'

In [ ]:
train_df.isnull().sum().max()

# No missing values

In [ ]:
print('# of classes = ', len(train_df['loss'].value_counts()))
train_df['loss'].value_counts()

In [ ]:
sns.set_theme(style="whitegrid")
sns.boxplot(data=train_df.loss)

In [ ]:
sns.displot(train_df.loss)

In [ ]:
print(train_df.dtypes.value_counts())
train_df.dtypes

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from catboost import CatBoostRegressor, CatBoostClassifier

In [ ]:
features = list(train_df.columns[1:101])
train_cat = np.zeros((250000,))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df[features], train_df['loss'])):
    X_train, X_valid = train_df.iloc[train_idx], train_df.iloc[valid_idx]
    y_train = X_train['loss']
    y_valid = X_valid['loss']
    X_train = X_train.drop('loss', axis=1)
    X_valid = X_valid.drop('loss', axis=1)
    
    cbr = CatBoostRegressor(random_state=42)

    cbr =  cbr.fit(X_train, y_train, verbose=False)
    temp_cat = cbr.predict(X_valid)
    train_cat[valid_idx] = temp_cat
    print(f'Fold {fold}: RMSE: ', mean_squared_error(y_valid, temp_cat, squared=False))
    
print(f'Cat-boost Accuracy: ', mean_squared_error(train_df['loss'], train_cat, squared=False))

In [ ]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2021/test.csv")
test_cat = np.zeros((len(test_df),))

X_test = test_df
temp_cat = cbr.predict(X_test)
test_cat= temp_cat

temp_cat = cbr.predict(X_test)


# **Sample submission**

In [ ]:
samp_sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
samp_sub['loss'] = test_cat
samp_sub.to_csv('cat_boost.csv', index = False)